**1. Import**

Let's start with imports and installing Hugging Face datasets.

In [ ]:
%%capture
!pip install datasets

In [ ]:
import torch
import numpy as np
from transformers import AutoTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling, AutoModelForCausalLM
from datasets import load_dataset, concatenate_datasets, Dataset, DatasetDict
from peft import LoraConfig, get_peft_model, TaskType

**2. Tokenization and Data Blocks**

Let's access our .txt files from Google Drive and prepare them with the tokenzier.

*   Step 1. Read all files as a concatenated text and Tokenize everything.
*   Step 2. Create blocks of data from the one long piece of tokens. (For GPT-2 models the block size is pretty limited due to their small 1024 token context window)
*   Step 3. Make Hugging Face dataset.



In [ ]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# set up tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# ---------Saber's path--------------

# read and tokenize each file
def load_and_tokenize(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    tokenized = tokenizer(text)
    return tokenized['input_ids']

# specify file path in Google Drive
data_path = "/content/drive/MyDrive/SEP775NLP_Final_Project/Friends_script/"

# run the function above to tokenize the data
train_ids = load_and_tokenize(f"{data_path}/cleaned_train_data.txt") # used latest version of processed data here
val_ids = load_and_tokenize(f"{data_path}/cleaned_val_data.txt")
test_ids = load_and_tokenize(f"{data_path}/cleaned_test_data.txt")
general_test_ids = load_and_tokenize(f"{data_path}/wikitext103_test.txt")

In [ ]:
# create blocks of sequences
block_size=512

def create_blocks(all_tokens, block_size):
  blocks = []
  # loop through all of the tokens, split them by block_size
  for i in range(0, len(all_tokens), block_size):
    tokens_in_this_block = all_tokens[i:i+block_size] # takes [0:512], then [512:1024]...
    blocks.append(tokens_in_this_block)
  return blocks # returns a list of list of blocks

# run the function above to create the data blocks
train_blocks = create_blocks(train_ids, block_size)
val_blocks = create_blocks(val_ids, block_size)
test_blocks = create_blocks(test_ids, block_size)
general_test_blocks = create_blocks(general_test_ids, block_size)

# verify that one block contains multiple lines of the script
print(train_blocks[0])

In [ ]:
# make dictionaries and then turn them into hugging face datasets
train_dataset = Dataset.from_dict({"input_ids": train_blocks})
val_dataset = Dataset.from_dict({"input_ids": val_blocks})
test_dataset = Dataset.from_dict({"input_ids": test_blocks})
general_test_dataset = Dataset.from_dict({"input_ids": general_test_blocks})

# create the Hugging Face dataset dict
dataset_dict = DatasetDict({"train": train_dataset, "validation": val_dataset, "test": test_dataset,
                            "general_test": general_test_dataset})

In [ ]:
# set up data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)  # For causal LM

**3. Model Training**

Let's set up the parameters for training here. We will also benchmark the pre-trained model's ability on a general Wiki language dataset (before fine-tuning). We will test the model again on this dataset after fine-tuning to see if our model retains the general language capability.

In [ ]:
# access the model from Hugging Face
model = AutoModelForCausalLM.from_pretrained("gpt2")
model.config.pad_token_id = model.config.eos_token_id

# --------------------------------------------------------------------------------(comment this section out if running without LoRA)
# freeze all the weights
#for param in model.parameters():
    #param.requires_grad = False
# -------------------------------------------------------------------------------- (this code might not be needed even when running LoRa, but just in case)

In [ ]:
# set up dummy trainer to benchmark the pre-trained model's capability on the general wiki test set
dummy_training_args = TrainingArguments(output_dir="./eval_output", do_train=False, per_device_eval_batch_size=4, report_to="none")
dummy_trainer = Trainer(model=model, args=dummy_training_args, eval_dataset=dataset_dict["general_test"], data_collator=data_collator)

# test the pre-trained model
benchmark_general_eval_results = dummy_trainer.evaluate(dataset_dict["general_test"])
print(f"General Wiki Data before Fine-Tuning: eval_loss = {benchmark_general_eval_results['eval_loss']:.2f}")

In [ ]:
# take a look at how many steps will be one epoch
num_of_train_blocks=(len(train_blocks))
step_per_epoch = num_of_train_blocks // 4 # note 4 here is the batch_size
print(step_per_epoch)

In [ ]:
# --------------------------------------------------------------------------------(comment this section out if running without LoRA)
# setup LoRA configuration and wrap the model with it
#lora_config = LoraConfig(r=32, lora_alpha=32, lora_dropout=0.05, bias="none", task_type=TaskType.CAUSAL_LM) # increase r for a more powerful adapter
#model = get_peft_model(model, lora_config)
#model.print_trainable_parameters()# verify we're only training LoRA weights
# --------------------------------------------------------------------------------

# set up training_args
training_args = TrainingArguments(output_dir="/content/drive/MyDrive/SEP775NLP_Final_Project/Models/gpt2-xl",
                                  evaluation_strategy="steps",
                                  save_strategy="steps",
                                  max_steps=2000,
                                  eval_steps=400,
                                  save_steps=400,
                                  logging_steps=400,
                                  #num_train_epochs=3,
                                  per_device_train_batch_size=4,
                                  per_device_eval_batch_size=4,
                                  learning_rate=8e-5,
                                  #save_total_limit=1,
                                  resume_from_checkpoint=True,
                                  load_best_model_at_end=True,
                                  metric_for_best_model="eval_loss",
                                  weight_decay=0.01, # added regularization
                                  warmup_steps=5, # added lr scheduler and warmup
                                  lr_scheduler_type="linear",
                                  report_to="none" # disable wandb
                                  ) # default optimizer is adamw

# setup trainer
trainer = Trainer(model=model,
                  args=training_args,
                  train_dataset=dataset_dict["train"],
                  eval_dataset=dataset_dict["validation"],
                  data_collator=data_collator)

In [ ]:
# start training
trainer.train()

**4. Evaluation**

Take the trained model, and evaluate its performance on the two different tests sets.

*   Test 1. Unseen FRIENDS scripts - to test model's ability to pick up the specific style of the show.
*   Test 2. General Wiki English data - to test model's general language capabilities. (compare to benchmark)



In [ ]:
# evaluation process
eval_results = trainer.evaluate(dataset_dict["test"])
general_eval_results = trainer.evaluate(dataset_dict["general_test"])

print(f"Unseen FRIENDS scripts: eval_loss = {eval_results['eval_loss']:.2f}")
print(f"General Wiki Data after Fine-Tuning: eval_loss = {general_eval_results['eval_loss']:.2f}")

**5. Generation**

Finally, we'll use this model to generate a new episode to see for ourselves how the model performs. We trialed the following different approaches for text generation and found that better results can be obtained with the last approach, so in this version of the code we only kept that approach.

*   Approach 1. Greedy Search (using argmax)
*   Approach 2. MultiNomial selection
*   Approach 3. MultiNomial selection with top_k and top_p filters

Updated the code not to end episode until certain amount of tokens have been generated. Also updated the code to stop after hitting the end of an episode.



In [ ]:
# create initial prompt and tokenize it
prompt = """
The One with students from McMaster University
Written by: GPT2-Base

[Scene: Monica's Apartment. Monica, Ross, Rachel, and Chandler are there.]

Monica: Hey guys! Big news—I heard there's going to be a party at McMaster University.

Ross: A party?

Rachel: Oh yeah! The students are celebrating their SEP775 class with the professors!

Chandler: We should join them! I mean, where is the fun in a party without your favorite FRIENDS characters?

(They all leave for the party.)

[Scene: SEP775 classroom at McMaster University. Rachel, Chandler, Ross, Monica, and Phoebe are hanging out with the students.]

Rachel: Wow, this place kicks ass! Look, isn’t that Professor Mahyar who teaches Natural Language Processing?

"""

model_inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=True).to(model.device)
model.generation_config.pad_token_id = tokenizer.pad_token_id # suppress padding warning

# create a list that holds the prompt, future tokens will be appended into this list later on
all_tokens = model_inputs["input_ids"][0].tolist()

# to avoid hitting memory limits, generate only one batch with the initial prompt, take last few tokens out to generate the next batch.
context_window = 700 # this is the sliding window - take this many tokens to set up next batch's generation
max_token = 2500
step_size = 128 # generate this many tokens at a time
num_of_loops = max_token // step_size

# end episode management
min_tokens_before_end = 2000 # generate this many tokens before being allowed to generate the word "end"
end_id = tokenizer.encode("end", add_special_tokens=False) # token_id of the word "end" (I checked it previously and it is only 1 token id, meaning it is the standalone word "end")
end_of_episode_marker = "### END OF EPISODE ###" # this marker was added during data pre-processing. it should come right after the "end" word

# generation loop
for i in range(num_of_loops):

  # suppresses the word "end" before reaching certain number of tokens
  if len(all_tokens) < min_tokens_before_end:
    bad_words = [end_id]
  else:
    bad_words = None

  output = model.generate(**model_inputs,
                          max_new_tokens=step_size,
                          do_sample=True,
                          top_k=40, # keep k highest probable options
                          top_p=0.75, # keep only the options that add up to p% probability, filter out the remaining less probable options
                          temperature=1, # to add a little bit more creativity if needed
                          repetition_penalty=1.2,
                          bad_words_ids=bad_words # bad words to generate before hitting certain number of tokens
                          )

  output_tokens = output[0].tolist()
  new_tokens = output_tokens[-step_size:] # get rid of the previous context and keep only new tokens

  new_text = tokenizer.decode(new_tokens, skip_special_tokens=True) # turn tokens to text to check if end marker has been generated
  start_index_of_end_marker = new_text.find(end_of_episode_marker) # returns -1 if the marker is not found, otherwise returns the start index

  if start_index_of_end_marker != -1: # if end marker is present
    truncated_new_text = new_text[: start_index_of_end_marker + len(end_of_episode_marker)] # get rid of what is after the end marker
    truncated_new_tokens = tokenizer.encode(truncated_new_text, add_special_tokens=False) # turn them into tokens
    all_tokens.extend(truncated_new_tokens) # and put them in the list
    break

  else:
      all_tokens.extend(new_tokens) # otherwise place the whole thing into the list without truncating

  previous_context = output_tokens[-context_window:] # set new previous_context for next loop
  previous_context_tensor = torch.tensor([previous_context]).to(model.device) # prepare the inputs and attention mask for next loop
  attention_mask = (previous_context_tensor != tokenizer.pad_token_id).long()
  model_inputs = {"input_ids": previous_context_tensor, "attention_mask":attention_mask}

# decode from the list and print results
generated_script = tokenizer.decode(all_tokens, skip_special_tokens=True)
print(generated_script)